In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import IPython.display as ipd
import librosa.display
import scipy 
import tensorflow as tf
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [ ]:
train_csv = pd.read_csv('/kaggle/input/audio-speech-sentiment/TRAIN.csv')
train_audio_path = "/kaggle/input/audio-speech-sentiment/TRAIN"
test_audio_path = "/kaggle/input/audio-speech-sentiment/TRAIN"

In [ ]:
def extract_features(file_name):
    audio,sample_rate = librosa.load(file_name,res_type="scipy")
    mfccs = librosa.feature.mfcc(y= audio,sr = sample_rate, n_mfcc = 40)
    return np.mean(mfccs.T, axis =0)

features = []
labels= []
for index,row in train_csv.iterrows():
    file_path = os.path.join(train_audio_path,row['Filename'])
    data = extract_features(file_path)
    features.append(data)
    labels.append(row['Class'])

In [ ]:
X = np.array(features)
le = LabelEncoder()
y = to_categorical(le.fit_transform(labels))
X_train,X_test,Y_train,Y_test = train_test_split(X,y, test_size = 0.2,random_state = 42)

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Input(shape=(40,1,1)),
    tf.keras.layers.Conv2D(32,(3,1),activation = "relu"), tf.keras.layers.MaxPooling2D(pool_size=(2,1)),tf.keras.layers.Flatten(),tf.keras.layers.Dense(3,activation="softmax")])


In [ ]:
X_train = X_train.reshape(X_train.shape[0],40,1,1)
X_test= X_test.reshape(X_test.shape[0],40,1,1)
model.compile(optimizer= "adam",loss = "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train,Y_train,epochs = 30,batch_size = 32)

In [ ]:
loss,accuracy= model.evaluate(X_test,Y_test)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred,axis = 1) 
y_true  = np.argmax(Y_test,axis =1)

print(classification_report(y_true,y_pred_classes,target_names= le.classes_))

In [ ]:
model.save('stress_detection_model.h5')